# Setting the file path

In [ ]:
import os
os.chdir("/content/drive/MyDrive/drive/Supplemental Material/TAVER-Entity-Resolution-main/TAVER-entity resolution")

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install tensorflow-gpu==1.15.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 1 Load data and remove irrelevant data.

In [ ]:
import pandas as pd
import numpy as np
import string
import math
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore") 
import os

import torch
from sentence_transformers import util

In [ ]:
# Loading datasets
file_path = 'data/Structured/iTunes-Amazon/'
table_a_df = pd.read_csv(file_path + 'tableA.csv')
table_b_df = pd.read_csv(file_path + 'tableB.csv')

train_df = pd.read_csv(file_path + 'train.csv')
valid_df = pd.read_csv(file_path + 'valid.csv')
test_df = pd.read_csv(file_path + 'test.csv')

In [ ]:
table_a_df.head()

,id,Song_Name,Artist_Name,Album_Name,Genre,Price,CopyRight,Time,Released
0,0,Runaway Train,Cam,Welcome to Cam Country - EP,"Country , Music , Contemporary Country , Honky...",$ 0.99,2015 Sony Music Entertainment,3:01,31-Mar-15
1,1,Track 14,Omi,Me 4 U,"Pop/Rock , Music , Pop , Dance , R&B / Soul",Album Only,"2015 Ultra Records , LLC under exclusive licen...",3:41,NaN
2,2,Lips Are Movin,Meghan Trainor,Title,"Pop , Music , Pop/Rock , Dance , Rock , Teen Pop",$ 1.29,"2014 , 2015 Epic Records , a division of Sony ...",3:01,9-Jan-15
3,3,I Want a Hippopotamus for Christmas,A Great Big World,I 'll Be Home For Christmas,"Holiday , Music",$ 1.29,"Compilation ( P ) 2014 Epic Records , a divisi...",2:20,24-Nov-14
4,4,Credit,Meghan Trainor,Title ( Deluxe ),"Pop , Music , Rock , Pop/Rock , Dance , Teen Pop",$ 1.29,"2014 , 2015 Epic Records , a division of Sony ...",2:51,9-Jan-15


In [ ]:
train_df.head()

,ltable_id,rtable_id,label
0,1409,51560,0
1,1289,34035,0
2,2666,35475,1
3,1157,27989,0
4,3332,7412,0


In [ ]:
def DataPeocess(attr_value):
    '''
    对输入的文本，提取当中的英文字母、浮点数、整数，将其他字符替换为空格
    '''
    s_list = list(str(attr_value))
    max_len = len(s_list)
    for i in range(len(s_list)):
        if s_list[i] == '.':
            if i < max_len-1 and i > 0 and s_list[i+1].isdigit() and s_list[i-1].isdigit():
                None
            else :
                s_list[i] = ' '
        elif not s_list[i].isalnum():
            s_list[i] = ' '
        else:
            s_list[i] == ' '

    return ''.join(s_list)

def ExtractData(df):
    '''
    对输入的dataFrame的所有属性值，去除无关字符，然后全部转换为小写
    '''
    attr_list = df.columns.values.tolist()
    attr_list.remove('id')

    
    for attr in attr_list:
        temp_w_num = 0

        for i in range(df.shape[0]):
            temp_value = DataPeocess(df.loc[i,attr])
            
            df.loc[i,attr] = temp_value.lower()

    return df

table_a_df = ExtractData(table_a_df)
table_b_df = ExtractData(table_b_df)

In [ ]:
attr_maxnum_d = dict()

attr_list = table_a_df.columns.values.tolist()
attr_list.remove('id')

for attr in attr_list:
    temp_num = 0

    for i in range(table_a_df.shape[0]):
        temp_value = table_a_df.loc[i, attr]
        temp_num = max(temp_num, len(temp_value.split()))

    for i in range(table_b_df.shape[0]):
        temp_value = table_b_df.loc[i, attr]
        temp_num = max(temp_num, len(temp_value.split()))

    attr_maxnum_d[attr] = temp_num

print(attr_maxnum_d.items())

dict_items([('Song_Name', 38), ('Artist_Name', 46), ('Album_Name', 32), ('Genre', 19), ('Price', 3), ('CopyRight', 37), ('Time', 3), ('Released', 3)])


# 2 Constructing sentences embedding model

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
attr_list = table_a_df.columns.values.tolist()[1:]
print(attr_list)

all_sentences = []

df_value = table_a_df[attr_list].values
for item in df_value:
    temp_list = list(item)
    temp_s = ', '.join(temp_list)
    all_sentences.append(temp_s)

df_value = table_b_df[attr_list].values
for item in df_value:
    temp_list = list(item)
    temp_s = ', '.join(temp_list)
    all_sentences.append(temp_s)

print(len(all_sentences))
print(all_sentences[0])

['Song_Name', 'Artist_Name', 'Album_Name', 'Genre', 'Price', 'CopyRight', 'Time', 'Released']
62830
runaway train, cam, welcome to cam country   ep, country   music   contemporary country   honky tonk,   0.99, 2015 sony music entertainment, 3 01, 31 mar 15


# 3 training sentences model

In [ ]:
from transformers import logging

logging.set_verbosity_warning()

In [ ]:
from sentence_transformers import SentenceTransformer, LoggingHandler
from sentence_transformers import models, util, datasets, evaluation, losses
from torch.utils.data import DataLoader

# Define your sentence transformer model using CLS pooling
model_name = 'bert-base-uncased'
word_embedding_model = models.Transformer(model_name)
# polling ：cls, mean, max
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), 'mean')
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

# Define a list with sentences (1k - 100k sentences)
train_sentences = all_sentences

# Create the special denoising dataset that adds noise on-the-fly
train_dataset = datasets.DenoisingAutoEncoderDataset(train_sentences)

# DataLoader to batch your data
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Use the denoising auto-encoder loss
train_loss = losses.DenoisingAutoEncoderLoss(model, decoder_name_or_path=model_name, tie_encoder_decoder=True)

## Call the fit method
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=100,
    weight_decay=0,
    scheduler='constantlr',
    optimizer_params={'lr': 1e-5},
    show_progress_bar=True
)

# model.save('./tsdae-model50')

In [ ]:
def TupleEmbedding(all_sentences, sentence_embedding_model):
    '''
    all_sentences: 包含所有元组句子的列表
    sentence_embedding_model: 将句子转换为句子向量的模型
    founction：将所有的句子，转换为对应的sentence vector
    '''
    sentences_vector = []
    for sen in all_sentences:
        sentences_vector.append(sentence_embedding_model.encode(sen))
    return np.array(sentences_vector)

In [ ]:
sentences_vector = TupleEmbedding(all_sentences, model)

train_df['similar'] = 0.0

for i in range(train_df.shape[0]):
    a_id = train_df.loc[i, 'ltable_id']
    b_id = train_df.loc[i, 'rtable_id']
    em_a = sentences_vector[a_id]
    em_b = sentences_vector[table_a_df.shape[0] + b_id]
    train_df.loc[i, 'similar'] = float(util.cos_sim(em_a, em_b))

train_df_0 = train_df[train_df['label'] == 0].copy(deep=True)
train_df_1 = train_df[train_df['label'] == 1].copy(deep=True)

# 4 Save the Embedding vector to a file

In [ ]:
X = sentences_vector
with open(file_path + 'X_array.txt', 'w') as fp:
  for item in X:
    temp = [str(i) for i in item]
    s = ' '.join(temp)
    fp.write(s + '\n')

In [ ]:
train_df_0.describe()

In [ ]:
train_df_1.describe()